# WIP: Robustness and Bias checks with DaCy
----

These falls into a larger category of [behavioral testing](https://github.com/marcotcr/checklist) of NLP systems.

## Setup
---
Before we start we assume you have installed DaCy and SpaCy if not you can run the following:


In [1]:
!pip install git+https://github.com/KennethEnevoldsen/DaCy

In [ ]:
# when running from a downloaded version of the github you will need to move back one folder.
import os
os.chdir("..")


----

Let's start of by loading DaCy as well as the smallest of the two models:

In [1]:
import dacy

# to see available models
for model in dacy.models():
    print(model)

# loading the smallest model
nlp = dacy.load("da_dacy_large_tft-0.0.0")

da_dacy_medium_tft-0.0.0
da_dacy_large_tft-0.0.0


# ADD MORE HERE (currently just a design)
the following is just how it is intended to work:

In [ ]:
train, dev, test = dane(predefined_splits=True) # read dataset

In [ ]:
# create apply functions

# for standard spacy pipeline
def apply_model(example):
    example.predicted = nlp(example.predicted.text)
    return example

# an example of a custom one for DaNLP BERT - maybe create a wrapper for this type of model?
def apply_bert_model(example):
    doc = example.predicted
    # uses spacy tokenization
    tokens, labels = bert.predict([t.text for t in example.predicted])
    ent = []
    for i, t in enumerate(zip(doc, labels)):
        token, label = t

        # turn OOB labels into spans
        if label == "O":
            continue
        iob, ent_type = label.split("-")
        if (i - 1 >= 0 and iob == "I" and labels[i - 1] == "O") or (
            i == 0 and iob == "I"
        ):
            iob = "B"
        if iob == "B":
            start = i
        if i + 1 >= len(labels) or labels[i + 1].split("-")[0] != "I":
            ent.append(Span(doc, start, i + 1, label=ent_type))
    doc.set_ents(ent)
    example.predicted = doc
    return example

In [ ]:
from spacy.scorer import Scorer
from spacy.training.augment import create_lower_casing_augmenter

scorer = Scorer()

out = dacy.score(corpus=test, # the corpus you want to apply it to
           augmenter=create_lower_casing_augmenter(1), # 1 = lowercase all
           apply_fn=apply_model # denne syntax er lidt grim, men ved ikke rigtig hvordan man skulle fikse det - evt. bare en spacy pipeline eller en apply func?
           score_fn=scorer.score, # scoring funciton this includes all - use partial(scorer.score_spans, attr ="ents") for only entities
           )
# tænker både augmenters, apply_fn og corpus can være en liste

Output structure:
```
>>> out
{"corpus1": {"augmenter1": {"model/apply_fn1": score_dict}}, "corpus2": ...}
```

men måske bliver det for nested. er det federe bare ikke at kunne inputte lister og så få noget alla:
```
score_dict
```

Det ville gøre den meget nemt at bruge, men du skulle hele tiden wrappe et loop omkring det. Den første løsning kunne man endda nemt lave en "to dataframe" converter.

In [ ]:

Example = (doc_gold, doc_predicted)

# unweighted
create_entity_augmenter(replace_dict = {"PER": [danske, kvinder, ikke-vestlige,...]})
    # den her går vi med indtil videre (efter samtale med KLN)


# weighted
create_entity_augmenter(replace_dict = {"PER": [(NAVN, weight), (NAVN, weight)]}, p = 1) # p how often should it replace a name (sampled from replace dict according to weight)

[orignal name, replace name, category of replace name, performance]